### RAG 

We are going to create RAG based Chat Agent which take input from user and provide answare based on the context provided by vectore search.

you requare following libraries <br><br>
`chromadb`
`sentence_transformers `
`ollama`
`pandas`

In [1]:
import chromadb
from sentence_transformers import SentenceTransformer
import pandas as pd
import ollama
from pprint import pprint

d:\machine_learning\test_projects\genai\chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chroma_client = chromadb.Client(
    settings=chromadb.Settings(
        persist_directory="./chroma_store"
    )
)

In [3]:
collection = chroma_client.get_or_create_collection(name="article")

In [4]:
news = pd.read_csv('article.csv')

In [12]:
news.shape

(10892, 10)

In [5]:
sub_news = news.iloc[0:200,:]

In [11]:
sub_news.shape

(200, 10)

In [18]:
sub_news.head(2)

,publishdate,createddate,modifieddate,section,author,tags,title,description,text,url
0,2020-08-11T17:55:12+05:30,2020-08-11T17:55:12+05:30,2020-08-11T17:55:12+05:30,National,Special Correspondent,"['flood-hit States', 'ration at doorsteps', 'R...",Ramvilas Paswan asks States to do doorstep rat...,"Parts of Bihar, Kerala, Karnataka, Assam and o...",Union Food Minister Ramvilas Paswan has appeal...,https://www.thehindu.com/news/national/ramvila...
1,2020-08-11T17:55:12+05:30,2020-08-11T17:55:12+05:30,2020-08-11T17:55:12+05:30,National,Special Correspondent,"['flood-hit States', 'ration at doorsteps', 'R...",Ramvilas Paswan asks States to do doorstep rat...,"Parts of Bihar, Kerala, Karnataka, Assam and o...",Union Food Minister Ramvilas Paswan has appeal...,https://www.thehindu.com/news/national/ramvila...


In [29]:
print(sub_news['text'][82])

As 
 on August 10, there is still no clarity on the way forward with regard to appointing a full-time president.
Also read: 
If the party extends 
, as is likely, it will have to officially inform the Election Commission of India (ECI) and subsequently get it endorsed by the Congress Working Committee (CWC).
The CWC, however, is unlikely to meet any time soon as all the attention is on the 
and 
 to prove majority on the floor of the House.
“The future course of action, as and when it is decided, will be shared with you”, Congress spokesperson Pawan Khera told reporters at a virtual press conference on Friday.
Also read: 
The party’s plan of a revamp, including appointment of a fulltime president is now caught between the Sachin Pilot-led rebellion in Rajasthan and the ongoing pandemic. 
Though there is a growing clamour among a section of party leaders for Rahul Gandhi to take over as the party president once again, he has not shown any urgency to get back to his earlier position even

In [6]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [7]:
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "mistral"

In [8]:
sub_news['text'].isnull().sum()

np.int64(5)

In [9]:
sub_news = sub_news.dropna(subset=['text'])

In [10]:
sub_news['text'].isnull().sum()

np.int64(0)

In [11]:
articles = sub_news['text'].to_list()

In [88]:
len(articles)

195

In [12]:
ids = [f"id{x}" for x in range(len(articles))]

In [90]:
len(ids)

195

In [13]:
embeddings = embedding_model.encode(
articles,
normalize_embeddings=True
).tolist()

In [14]:
len(embeddings)

195

In [15]:
collection.add(
    documents=articles,
    embeddings=embeddings,
    ids=ids
)

In [16]:
def retrieve_context(query, k=3):
    query_embedding = embedding_model.encode(
        query,
        normalize_embeddings=True
    ).tolist()

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k
    )

    return results["documents"][0]


In [ ]:
def build_prompt(context, question):
    context_text = "\n".join(context)

    prompt = f"""
    You are a helpful assistant.
    Answer the question using ONLY the context below.
    If the answer is not in the context, say "I don't know".

    Context:
    {context_text}

    Question:
    {question}

    Answer:
    """
    return prompt


In [18]:

def generate_answer(prompt):
    response = ollama.chat(
        model="mistral",
        messages=[
            {"role": "user", "content": prompt},
            
        ],
        options={
            "num_gpu": 0 
        }
    )
    return response["message"]["content"]


In [19]:
def rag_pipeline(question):
    context = retrieve_context(question)
    prompt = build_prompt(context, question)
    answer = generate_answer(prompt)
    return answer


In [20]:
question = "who is Congress chief spokesperson?"
response = rag_pipeline(question)
print(response)

 Randeep Surjewala
